In [158]:
%matplotlib inline
import os
import subprocess
from typing import Tuple, List
import os.path as osp
import time
import pickle

from PIL import Image
from pyquaternion import Quaternion
from nuscenes import NuScenes
from nuscenes.utils.data_classes import Box
from nuscenes.utils.geometry_utils import points_in_box
from nuscenes.utils.geometry_utils import view_points
from nuscenes.utils.geometry_utils import BoxVisibility
from nuscenes.panoptic.panoptic_utils import paint_panop_points_label
from nuscenes.lidarseg.lidarseg_utils import paint_points_label
from nuscenes.utils.data_classes import LidarPointCloud, RadarPointCloud

import numpy as np
from skimage.feature import peak_local_max
from scipy.spatial import ConvexHull, convex_hull_plot_2d

import cv2

nusc = NuScenes(version='v1.0-trainval', dataroot='/data/datasets/nuscenes', verbose=True)


Loading NuScenes tables for version v1.0-trainval...
Loading nuScenes-panoptic...
32 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
34149 panoptic,
Done loading in 49.213 seconds.
Reverse indexing ...
Done reverse indexing in 6.1 seconds.


In [159]:
def normalize(smap):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    # This operator uses the global
    # maximum of each map to obtain a dynamic range between
    # 0 and 1
   
    if smap.min() <0 :
        sal_map=smap+np.abs(smap.min())
    else:
        sal_map=smap
        
    sal_map=sal_map/sal_map.max()
    
    return sal_map
    
def normalized_and_sum(smap1, smap2):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    
    n_smap1 = normalize(smap1)
    n_smap2 = normalize(smap2)
    
    raw_sum_map = n_smap1 + n_smap2
    
    outmap = normalize(raw_sum_map)
    
    return outmap


def normalized_and_maximum(smap1, smap2):
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    
    n_smap1 = normalize(smap1)
    n_smap2 = normalize(smap2)
    
    outmap = np.maximum(n_smap1, n_smap2)

    return outmap
    
def simple_pixel_wise_product(map_a, map_b):
    comb_map=map_a*map_b
    
    #normalize
    comb_map=normalize(comb_map)
    
    return comb_map

def coherent_normalization_with_weighted_sum(smap1, smap2, alpha=0.5, beta=1.0):
    # REF: https://ieeexplore.ieee.org/document/6605606
    
    outmap = (1 - alpha)*smap1 + alpha*smap2 + beta*( smap1*smap2)
    
    return normalize(outmap)

def average_local_maxima(smap):
    # t = time.time()
    local_max_loc=peak_local_max(smap)
    # print("peak local max {:0.10f}".format(time.time()-t))
    # t = time.time()

    #all local maxima
    all_local_max = smap[local_max_loc[:,0], local_max_loc[:,1]]
    # print("all local max {:0.10f}".format(time.time()-t))
    # t = time.time()

    # drop the absolute maximum
    other_local_max = np.delete(all_local_max, np.argmax(all_local_max))
    # print("drop the absolute maximum max {:0.10f}".format(time.time()-t))
    # t = time.time()

    mean = np.mean(other_local_max)
    # print("mean in average local maxima {:0.10f}".format(time.time()-t))
    # t = time.time()
    return mean

def global_nonlinear_amplification(smap1, smap2):
    # original REF: http://authors.library.caltech.edu/71459/1/161_1.pdf
    # REF: https://ieeexplore.ieee.org/abstract/document/5651381
    # REF for local maxima: https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_peak_local_max.html
    
    # t = time.time()
    n_smap1 = normalize(smap1)
    # print("normalize 1 maximum max {:0.10f}".format(time.time()-t))
    # t = time.time()
    n_smap2 = normalize(smap2)
    # print("normalize 2 maximum max {:0.10f}".format(time.time()-t))
    # t = time.time()

    # follows notation of Le Meur 
    M_1 = n_smap1.max()
    # print("max of normalized saliency 1 {:0.10f}".format(time.time()-t))
    # t = time.time()
    m_1 = average_local_maxima (smap1)
    # print("average local maxima total 1 {:0.10f}".format(time.time()-t))
    # t = time.time()


    M_2 = n_smap2.max()
    # print("max of normalized saliency 2 {:0.10f}".format(time.time()-t))
    # t = time.time()

    m_2 = average_local_maxima (smap2)
    # print("average local maxima total 2 {:0.10f}".format(time.time()-t))
    # t = time.time()
    
    outmap = ( n_smap1 * (M_1-m_1)**2 ) + ( n_smap2 * (M_2-m_2 )**2 )
    # print("outmap {:0.10f}".format(time.time()-t))
    # t = time.time()

    return outmap


In [160]:
font = cv2.FONT_HERSHEY_COMPLEX
font_scale = 0.5
thickness = 1


def cv2_put_multi_line_text(im: np.ndarray,
                            text: str,
                            center: Tuple,
                            color: Tuple,
                            font=cv2.FONT_HERSHEY_COMPLEX,
                            font_scale=0.5,
                            thickness=1) -> None:
    text_size, _ = cv2.getTextSize("dummy_text", font, font_scale, thickness)
    line_height = text_size[1] + 5
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, color[::-1], thickness, cv2.LINE_AA)

In [161]:
def map_pointcloud_to_box(nusc: NuScenes,
                        sample_token: str,
                        pointsensor_token: str,
                        camera_token: str,
                        box: Box,
                        # point_sensor_channel: str ='LIDAR_TOP',
                        # camera_channel: str = 'CAM_FRONT',
                        min_dist: float = 1.0,
                        render_intensity: bool = False,
                        show_lidarseg: bool = False,
                        filter_lidarseg_labels: List = None,
                        lidarseg_preds_bin_path: str = None,
                        show_panoptic: bool = False) -> Tuple:
    """
    Given a point sensor (lidar/radar) token and camera sample_data token, load pointcloud and map it to the image
    plane.
    :param nusc: Active NuScenes object.
    :param sample_token: Sample token.
    :param pointsensor_token: Lidar/radar sample_data token.
    :param camera_token: Camera sample_data token.
    :param box: Box that we want map points to.
    :param min_dist: Distance from the camera below which points are discarded.
    :param render_intensity: Whether to render lidar intensity instead of point depth.
    :param show_lidarseg: Whether to render lidar intensity instead of point depth.
    :param filter_lidarseg_labels: Only show lidar points which belong to the given list of classes. If None
        or the list is empty, all classes will be displayed.
    :param lidarseg_preds_bin_path: A path to the .bin file which contains the user's lidar segmentation
                                    predictions for the sample.
    :param show_panoptic: When set to True, the lidar data is colored with the panoptic labels. When set
        to False, the colors of the lidar data represent the distance from the center of the ego vehicle.
        If show_lidarseg is True, show_panoptic will be set to False.
    :return (pointcloud <np.float: 2, n)>, coloring <np.float: n>, image <Image>).
    """

    pointsensor = nusc.get('sample_data', pointsensor_token)
    cam = nusc.get('sample_data', camera_token)
    pcl_path = osp.join(nusc.dataroot, pointsensor['filename'])
    if pointsensor['sensor_modality'] == 'lidar':
        if show_lidarseg or show_panoptic:
            gt_from = 'lidarseg' if show_lidarseg else 'panoptic'
            assert hasattr(nusc.nusc, gt_from), f'Error: nuScenes-{gt_from} not installed!'

            # Ensure that lidar pointcloud is from a keyframe.
            assert pointsensor['is_key_frame'], \
                'Error: Only pointclouds which are keyframes have lidar segmentation labels. Rendering aborted.'

            assert not render_intensity, 'Error: Invalid options selected. You can only select either ' \
                                            'render_intensity or show_lidarseg, not both.'

        pc = LidarPointCloud.from_file(pcl_path)
    else:
        pc = RadarPointCloud.from_file(pcl_path)
    # im = Image.open(osp.join(nusc.dataroot, cam['filename']))

    # Points live in the point sensor frame. So they need to be transformed via global to the image plane.
    # First step: transform the pointcloud to the ego vehicle frame for the timestamp of the sweep.
    cs_record = nusc.get('calibrated_sensor', pointsensor['calibrated_sensor_token'])
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
    pc.translate(np.array(cs_record['translation']))

    # Second step: transform from ego to the global frame.
    poserecord = nusc.get('ego_pose', pointsensor['ego_pose_token'])
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
    pc.translate(np.array(poserecord['translation']))

    # Third step: transform from global into the ego vehicle frame for the timestamp of the image.
    poserecord = nusc.get('ego_pose', cam['ego_pose_token'])
    pc.translate(-np.array(poserecord['translation']))
    pc.rotate(Quaternion(poserecord['rotation']).rotation_matrix.T)

    # Fourth step: transform from ego into the camera.
    cs_record = nusc.get('calibrated_sensor', cam['calibrated_sensor_token'])
    pc.translate(-np.array(cs_record['translation']))
    pc.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)

    # Fifth step: actually take a "picture" of the point cloud.
    # Grab the depths (camera frame z axis points away from the camera).
    depths = pc.points[2, :]

    if render_intensity:
        assert pointsensor['sensor_modality'] == 'lidar', 'Error: Can only render intensity for lidar, ' \
                                                            'not %s!' % pointsensor['sensor_modality']
        # Retrieve the color from the intensities.
        # Performs arbitary scaling to achieve more visually pleasing results.
        intensities = pc.points[3, :]
        intensities = (intensities - np.min(intensities)) / (np.max(intensities) - np.min(intensities))
        intensities = intensities ** 0.1
        intensities = np.maximum(0, intensities - 0.5)
        coloring = intensities
    elif show_lidarseg or show_panoptic:
        assert pointsensor['sensor_modality'] == 'lidar', 'Error: Can only render lidarseg labels for lidar, ' \
                                                            'not %s!' % pointsensor['sensor_modality']

        gt_from = 'lidarseg' if show_lidarseg else 'panoptic'
        semantic_table = getattr(nusc.nusc, gt_from)

        if lidarseg_preds_bin_path:
            sample_token = nusc.get('sample_data', pointsensor_token)['sample_token']
            lidarseg_labels_filename = lidarseg_preds_bin_path
            assert os.path.exists(lidarseg_labels_filename), \
                'Error: Unable to find {} to load the predictions for sample token {} (lidar ' \
                'sample data token {}) from.'.format(lidarseg_labels_filename, sample_token, pointsensor_token)
        else:
            if len(semantic_table) > 0:  # Ensure {lidarseg/panoptic}.json is not empty (e.g. in case of v1.0-test).
                lidarseg_labels_filename = osp.join(nusc.dataroot,
                                                    nusc.get(gt_from, pointsensor_token)['filename'])
            else:
                lidarseg_labels_filename = None

        if lidarseg_labels_filename:
            # Paint each label in the pointcloud with a RGBA value.
            if show_lidarseg:
                coloring = paint_points_label(lidarseg_labels_filename,
                                                filter_lidarseg_labels,
                                                nusc.lidarseg_name2idx_mapping,
                                                nusc.colormap)
            else:
                coloring = paint_panop_points_label(lidarseg_labels_filename,
                                                    filter_lidarseg_labels,
                                                    nusc.lidarseg_name2idx_mapping,
                                                    nusc.colormap)

        else:
            coloring = depths
            print(f'Warning: There are no lidarseg labels in {nusc.version}. Points will be colored according '
                    f'to distance from the ego vehicle instead.')
    else:
        # Retrieve the color from the depth.
        coloring = depths

    # Take the actual picture (matrix multiplication with camera-matrix + renormalization).
    points = view_points(pc.points[:3, :], np.array(cs_record['camera_intrinsic']), normalize=True)

    # Remove points that are either outside or behind the camera. Leave a margin of 1 pixel for aesthetic reasons.
    # Also make sure points are at least 1m in front of the camera to avoid seeing the lidar points on the camera
    # casing for non-keyframes which are slightly out of sync.
    mask = np.ones(depths.shape[0], dtype=bool)
    mask = np.logical_and(mask, depths > min_dist)
    # mask = np.logical_and(mask, points[0, :] > 1)
    # mask = np.logical_and(mask, points[0, :] < im.size[0] - 1)
    # mask = np.logical_and(mask, points[1, :] > 1)
    # mask = np.logical_and(mask, points[1, :] < im.size[1] - 1)
    height = cam['height']
    width = cam['width']
    mask = np.logical_and(mask, points[0, :] > 0)
    mask = np.logical_and(mask, points[0, :] < width)
    mask = np.logical_and(mask, points[1, :] > 0)
    mask = np.logical_and(mask, points[1, :] < height)
    # Also only take only points inside the box into account
    mask = np.logical_and(mask, points_in_box(box, pc.points[:3, :]))
    
    points = points[:, mask]
    coloring = coloring[mask]

    return points, coloring #, im


def render_pointcloud_to_box(im: np.ndarray, 
                             nusc: NuScenes,
                             sample_token: str,
                             pointsensor_token: str,
                             camera_token: str,
                             box: Box,
                             # point_sensor_channel: str ='LIDAR_TOP',
                             # camera_channel: str = 'CAM_FRONT',
                             min_dist: float = 1.0,
                             render_intensity: bool = False,
                             show_lidarseg: bool = False,
                             filter_lidarseg_labels: List = None,
                             lidarseg_preds_bin_path: str = None,
                             show_panoptic: bool = False) -> Tuple:
    """
    Given a point sensor (lidar/radar) token and camera sample_data token, load pointcloud and map it to the image
    plane.
    :param nusc: Active NuScenes object.
    :param sample_token: Sample token.
    :param pointsensor_token: Lidar/radar sample_data token.
    :param camera_token: Camera sample_data token.
    :param box: Box that we want map points to.
    :param min_dist: Distance from the camera below which points are discarded.
    :param render_intensity: Whether to render lidar intensity instead of point depth.
    :param show_lidarseg: Whether to render lidar intensity instead of point depth.
    :param filter_lidarseg_labels: Only show lidar points which belong to the given list of classes. If None
        or the list is empty, all classes will be displayed.
    :param lidarseg_preds_bin_path: A path to the .bin file which contains the user's lidar segmentation
                                    predictions for the sample.
    :param show_panoptic: When set to True, the lidar data is colored with the panoptic labels. When set
        to False, the colors of the lidar data represent the distance from the center of the ego vehicle.
        If show_lidarseg is True, show_panoptic will be set to False.
    :return (pointcloud <np.float: 2, n)>, coloring <np.float: n>, image <Image>).
    """
    points, coloring = map_pointcloud_to_box(
        nusc, sample_token=sample_token, pointsensor_token=pointsensor_token, camera_token=camera_token, box=box, min_dist=min_dist,
        render_intensity=render_intensity, show_lidarseg=show_lidarseg, filter_lidarseg_labels=filter_lidarseg_labels,
        lidarseg_preds_bin_path=lidarseg_preds_bin_path, show_panoptic=show_panoptic)
    # Need to be finished later for more precise contours of tracked objects and new objects


In [162]:
def mask_pixels_from_box_cv2(box: Box,
                             width: int,
                             height: int,
                             view: np.ndarray = np.eye(3),
                             normalize: bool = False) -> None:
    """
    Renders box using OpenCV2.
    :param Box: Box to be rendered
    :param width: width of the image
    :param height: height of the image
    :param view: camera intrinsics
    :param normalize: Whether to normalize the remaining coordinate.
    """

    corners = view_points(box.corners(), view, normalize=normalize)[:2, :]  # all x, y picture coordinates

    # simply fill all 6 sides of the cuboid without differentiating which sides are in the fore- or background
    mask = np.zeros((height, width), dtype=np.uint8)
    front = np.ix_([0, 1, 2, 3], [0, 1])
    rear = np.ix_([4, 5, 6, 7], [0, 1])
    bottom = np.ix_([2, 3, 7, 6], [0, 1])
    top = np.ix_([0, 1, 5, 4], [0, 1])
    left = np.ix_([0, 3, 7, 4], [0, 1])
    right = np.ix_([1, 2, 6, 5], [0, 1])
    cv2.fillPoly(mask, pts=[corners.T[front].astype(int)], color=(255))
    cv2.fillPoly(mask, pts=[corners.T[rear].astype(int)], color=(255))
    cv2.fillPoly(mask, pts=[corners.T[bottom].astype(int)], color=(255))
    cv2.fillPoly(mask, pts=[corners.T[top].astype(int)], color=(255))
    cv2.fillPoly(mask, pts=[corners.T[left].astype(int)], color=(255))
    cv2.fillPoly(mask, pts=[corners.T[right].astype(int)], color=(255))

    pixels_tuple = np.where(mask == 255)
    return mask, pixels_tuple

In [163]:
gbvs_bin = "/home/serov/code/cpp/GBVS_fork/build/gbvs_main"

def get_gbvs_cpp(input_filename):
    output_filename = input_filename.replace('image.png', 'gbvs.exr')
    gbvs_cmd = gbvs_bin + " -i " + input_filename + " -o " + output_filename
    p = subprocess.Popen(gbvs_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.wait() == 0:
        return cv2.imread(output_filename, flags=(cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH) )
    else:
        raise Exception("Couldn't create GBVS saliency map.")

def create_gbvs_cpp(input_filename):
    output_filename = input_filename.replace('image.png', 'gbvs.exr')
    gbvs_cmd = gbvs_bin + " -i " + input_filename + " -o " + output_filename
    p = subprocess.Popen(gbvs_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.wait() == 0:
        print("Created gbvs map at:", output_filename)
    else:
        raise Exception("Couldn't create GBVS saliency map.")


In [164]:

def mean_absolute_error(saliency: np.ndarray,
                        groundtruth: np.ndarray) -> float:
    assert(saliency.shape == groundtruth.shape)
    height, width = saliency.shape
    dtype_saliency = saliency.dtype
    return 1 / (height * width) * np.sum(np.absolute(saliency - np.asarray(groundtruth / 255.0, dtype=dtype_saliency)))

def mean_absolute_error_new(saliency: np.ndarray,
                            mask: np.ndarray):
    assert(saliency.shape == mask.shape)
    height, width = saliency.shape
    dtype_saliency = saliency.dtype
    
    indices_new_obj = np.where(mask == 255)
    num_pixels_new = len(indices_new_obj[0])
    saliency_new = saliency[[indices_new_obj[0], indices_new_obj[1]]]
    mae_new_only = 1 / num_pixels_new * np.sum(np.absolute(saliency_new - 1))

    mae_new_im = 1 / (height * width) * np.sum(np.absolute(saliency - np.asarray(mask.clip(0, 1).astype(dtype_saliency)))) 
    
    return mae_new_only, mae_new_im

def mean_absolute_error_multi(saliency: np.ndarray,
                              mask_new: np.ndarray,
                              mask_already: np.ndarray) -> tuple:
    assert(saliency.shape == mask_new.shape)
    assert(mask_new.shape == mask_already.shape)
    height, width = saliency.shape
    dtype_saliency = saliency.dtype
    indices_new_obj = np.where(mask_new == 255)
    num_pixels_new = len(indices_new_obj[0])
    saliency_new = saliency[[indices_new_obj[0], indices_new_obj[1]]]

    mae_new = 1 / num_pixels_new * np.sum(np.absolute(saliency_new - 1))
    mae_already = 1 / (height * width) * np.sum(np.absolute(saliency - np.asarray(mask_already.clip(0, 1).astype(dtype_saliency))))
    
    return mae_new, mae_already


In [165]:
def overlay_image_and_saliency_as_heatmap(im: np.ndarray,
                                          saliency: np.ndarray,
                                          resize_factor: float) -> np.ndarray:

    im_resize = cv2.resize(im, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    saliency_resize = cv2.resize(saliency, None, fx=resize_factor, fy=resize_factor, interpolation=cv2.INTER_AREA)
    if im_resize.dtype != np.uint8:
        im_resize = np.asarray(im_resize, dtype=np.uint8)
    if saliency_resize.dtype != np.uint8:
        saliency_resize = np.asarray(saliency_resize * 255, dtype=np.uint8)
    if len(im_resize.shape) != len(saliency_resize.shape):
        if len(saliency_resize.shape) == 2:  # grayscale
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_GRAY2BGR)  # convert to color
        elif len(saliency_resize).shape == 3:  # color
            saliency_resize = cv2.cvtColor(saliency_resize, cv2.COLOR_BGR2GRAY)  # convert to grayscale

    saliency_resize = cv2.applyColorMap(saliency_resize, cv2.COLORMAP_JET)

    alpha = 0.7
    beta = 0.3
    gamma = 0
    return cv2.addWeighted(im_resize, alpha, saliency_resize, beta, gamma)


In [166]:
def convex_mask_and_hull_points_from_lidar_points(box_mask: np.ndarray,
                           lidar_points_inside_box: np.ndarray,
                           width: int,
                           height: int) -> Tuple:
    # check number of lidar points and overall size of the masked_box vs. the contour created from projected lidar points
    mask = np.zeros((height, width), dtype=np.uint8)
    
    hull_points = cv2.convexHull(lidar_points_inside_box[:2, :].T.astype(np.float32))
  
    if hull_points is None:
        hull_valid = False
    else:
        hull_valid = True
        # if scale != 1.0:
        #     M = cv2.moments(hull_points)
        #     cx = int(M['m10']/M['m00'])
        #     cy = int(M['m01']/M['m00'])
        #     hull_points_norm = hull_points - [cx, cy]
        #     hull_points_scaled = hull_points_norm * scale
        #     hull_points = hull_points_scaled + [cx, cy]
        cv2.fillConvexPoly(mask, hull_points.astype(np.int32), color=(255))

    return mask, hull_valid, hull_points

def scale_contour(contour: np.ndarray,
                  scale: float)-> np.ndarray:
    if scale == 1.0:
        return contour
    else:
        M = cv2.moments(contour)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        contour_norm = contour - [[cx], [cy]]
        contour_scaled = scale * contour_norm 
        # contour_scaled += [[cy], [cx]]
        return contour_scaled, (cy, cy)


def center_of_hull_points(hull_points: np.ndarray):
    M = cv2.moments(hull_points)
    if M['m00'] == 0:
        return (None, None)
    else:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        return (cx, cy)


In [167]:
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

def box_render_cv2_text(box: Box,
                        nusc: NuScenes,
                        im: np.ndarray,
                        view: np.ndarray = np.eye(3),
                        normalize: bool = False,
                        colors: Tuple = ((0, 0, 255), (255, 0, 0), (155, 155, 155)),
                        linewidth: int = 2,
                        render_text: bool = True) -> None:
    """
    Renders box using OpenCV2.
    :param Box: Box to be rendered
    :param nusc: Active NuScenes object
    :param im: <np.array: width, height, 3>. Image array. Channels are in BGR order.
    :param view: <np.array: 3, 3>. Define a projection if needed (e.g. for drawing projection in an image).
    :param is_key_frame
    :param normalize: Whether to normalize the remaining coordinate.
    :param colors: ((R, G, B), (R, G, B), (R, G, B)). Colors for front, side & rear.
    :param linewidth: Linewidth for plot.
    """
    corners = view_points(box.corners(), view, normalize=normalize)[:2, :]

    def draw_rect(selected_corners, color):
        prev = selected_corners[-1]
        for corner in selected_corners:
            cv2.line(im,
                     (int(prev[0]), int(prev[1])),
                     (int(corner[0]), int(corner[1])),
                     color, linewidth)
            prev = corner

    # Draw the sides
    for i in range(4):
        cv2.line(im,
                 (int(corners.T[i][0]), int(corners.T[i][1])),
                 (int(corners.T[i + 4][0]), int(corners.T[i + 4][1])),
                 colors[2][::-1], linewidth)

    # Draw front (first 4 corners) and rear (last 4 corners) rectangles(3d)/lines(2d)
    draw_rect(corners.T[:4], colors[0][::-1])
    draw_rect(corners.T[4:], colors[1][::-1])

    # Draw line indicating the front
    center_bottom_forward = np.mean(corners.T[2:4], axis=0)
    center_bottom = np.mean(corners.T[[2, 3, 7, 6]], axis=0)
    cv2.line(im,
             (int(center_bottom[0]), int(center_bottom[1])),
             (int(center_bottom_forward[0]), int(center_bottom_forward[1])),
             colors[0][::-1], linewidth)

    # Add some text to the bounding box
    # Create text to be written
    attribute_tokens =  nusc.get('sample_annotation', box.token)['attribute_tokens']
    text = box.name
    for attribute_token in attribute_tokens:
        attribute = nusc.get('attribute', attribute_token)
        text += '\n' + attribute['name']
    visibility = nusc.get('sample_annotation', box.token)['visibility_token']
    text += '\nvisibility ' + visibility
    # Setup multi line text
    font = cv2.FONT_HERSHEY_COMPLEX
    font_scale = 0.5
    thickness = 1
    text_size, _ = cv2.getTextSize(box.name, font, font_scale, thickness)
    line_height = text_size[1] + 5
    center = np.mean(corners.T[:], axis=0)
    y0 = int(center[1])
    for i, text_line in enumerate(text.split('\n')):
        y = y0 + i * line_height
        cv2.putText(im, text_line, (int(center[0]), y),
                    cv2.FONT_HERSHEY_COMPLEX, font_scale, colors[0][::-1], thickness, cv2.LINE_AA)



In [168]:
class RunningStats():
    """ 
    see https://github.com/liyanage/python-modules/blob/master/running_stats.py
    """
    def __init__(self):
        self.n = 0
        self.old_m = 0
        self.new_m = 0
        self.old_s = 0
        self.new_s = 0
        self.categories = []
        self.mean_absolute_error_new_objects = []
        self.mean_absolute_error_already_tracked_objects = []
        self.mean_absolute_error_whole_image = []

    def clear(self):
        self.n = 0

    def add_sample(self, x):
        self.n += 1

        if self.n == 1:
            self.old_m = self.new_m = x
            self.old_s = 0
        else:
            self.new_m = self.old_m + (x - self.old_m) / self.n
            self.new_s = self.old_s + (x - self.old_m) * (x - self.new_m)

            self.old_m = self.new_m
            self.old_s = self.new_s

    def mean(self):
        return self.new_m if self.n else 0.0

    def variance(self):
        return self.new_s / (self.n - 1) if self.n > 1 else 0.0

    def standard_deviation(self):
        return np.sqrt(self.variance())

class SaliencyEvalStats():
    def __init__(self, name) -> None:
        # arrays containing relevant data
        self.n = int(0)
        self.name = name
        self.mean_absolute_error_new_objects: float = []
        self.mean_absolute_error_already_tracked_objects: float = []
    
    def add_sample(self, mean_absolute_error_new_object, mean_absolute_error_already_tracked_object):
        self.n += 1
        self.mean_absolute_error_new_objects.append(mean_absolute_error_new_object)
        self.mean_absolute_error_already_tracked_objects.append(mean_absolute_error_already_tracked_object)

    def print_stats(self):
        print("{:30} n: {:d}  mae_new {:0.10f}  std_new  {:0.10f}  mae_tracked  {:0.10f}  std_tracked  {:0.10f}".format(
            self.name, self.n, np.mean(np.asarray(self.mean_absolute_error_new_objects)),
            np.std(np.asarray(self.mean_absolute_error_new_objects)),
            np.mean(np.asarray(self.mean_absolute_error_already_tracked_objects)),
            np.std(np.asarray(self.mean_absolute_error_already_tracked_objects))))

    def get_stats(self):
        return np.array([[np.mean(np.asarray(self.mean_absolute_error_new_objects)),
                          np.std(np.asarray(self.mean_absolute_error_new_objects))]
                         [np.mean(np.asarray(self.mean_absolute_error_already_tracked_objects)),
                          np.std(np.asarray(self.mean_absolute_error_already_tracked_objects))]])

    def n(self):
        return self.n


class SaliencyEvalNewStats():
    def __init__(self, name) -> None:
        # arrays containing relevant data
        self.n = int(0)
        self.name = name
        self.mean_absolute_error_new_objects: float = []
        self.mean_absolute_error_new_objects_whole_im: float = []

    def add_sample(self, mean_absolute_error_new_object, mean_absolute_error_new_object_whole_im):
        self.n += 1
        self.mean_absolute_error_new_objects.append(mean_absolute_error_new_object)
        self.mean_absolute_error_new_objects_whole_im.append(mean_absolute_error_new_object_whole_im)

    def print_stats(self):
        print("{:30} n: {:d}  mae_new {:0.10f}  std_new  {:0.10f}  mae_new_im  {:0.10f}  std_new_im  {:0.10f}".format(
            self.name, self.n, np.mean(np.asarray(self.mean_absolute_error_new_objects)),
            np.std(np.asarray(self.mean_absolute_error_new_objects)),
            np.mean(np.asarray(self.mean_absolute_error_new_objects_whole_im)),
            np.std(np.asarray(self.mean_absolute_error_new_objects_whole_im))))

    def get_stats(self):
        return np.array([[np.mean(np.asarray(self.mean_absolute_error_new_objects)),
                          np.std(np.asarray(self.mean_absolute_error_new_objects))]
                         [np.mean(np.asarray(self.mean_absolute_error_new_objects_whole_im)),
                          np.std(np.asarray(self.mean_absolute_error_new_objects_whole_im))]])

    def n(self):
        return self.n


In [169]:
def display_im_and_saliency(im: np.ndarray,
                            saliency: np.ndarray,
                            window_name: str,
                            # mask_new: np.ndarray,
                            # mask_already: np.ndarray,
                            # mae_new: float,
                            # mae_already: float
                            ) -> None:
    (height, width, channels) = im.shape
    saliency_heatmap = cv2.cvtColor((255*saliency).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    assert(im.shape == saliency_heatmap.shape)
    saliency_heatmap = cv2.applyColorMap(saliency_heatmap, cv2.COLORMAP_JET)
    display_im = cv2.addWeighted(im, 0.7, saliency_heatmap, 0.3, 0.0)
    cv2.imshow(window_name, display_im)
    cv2.waitKey(0)
    cv2.destroyWindow(window_name)

def debug_im_and_saliency(im: np.ndarray,
                          saliency: np.ndarray,
                          window_name: str,
                          mae_new: float,
                          mae_already: float
                          ) -> None:
    (height, width, channels) = im.shape
    saliency_heatmap = cv2.cvtColor((255*saliency).astype(np.uint8), cv2.COLOR_GRAY2BGR)
    assert(im.shape == saliency_heatmap.shape)
    saliency_heatmap = cv2.applyColorMap(saliency_heatmap, cv2.COLORMAP_JET)
    display_im = cv2.addWeighted(im, 0.7, saliency_heatmap, 0.3, 0.0)
    cv2_put_multi_line_text(display_im, "mae_new_only = {:0.10f}".format(mae_new), (width/2, height/2), BLACK)
    cv2_put_multi_line_text(display_im, "mae_new_im = {:0.10f}".format(mae_already), (width/2, height/2 + 20), BLACK)
    cv2.imshow(window_name, display_im)
    cv2.waitKey(0)
    cv2.destroyWindow(window_name)


In [170]:
camera_channel = 'CAM_FRONT'
pointsensor_channel = 'LIDAR_TOP'
minimum_visibility = 2 # excluding objects that are only visibile 0-40%
height = 900
width = 1600
break_all = False
save_already_tracked = False
debug = True
debug_single = True

data_path = "/data/datasets/nuscenes_results/birth_frames_and_masks"

# create spectral residual object
spectral_residual_h64w64 = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual_h64w64.setImageHeight(64)
spectral_residual_h64w64.setImageWidth(64)
spectral_residual_h128w128 = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual_h128w128.setImageHeight(128)
spectral_residual_h128w128.setImageWidth(128)
# h8w64
spectral_residual_h8w64 = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual_h8w64.setImageHeight(8)
spectral_residual_h8w64.setImageWidth(128)
# h8w128
spectral_residual_h8w128 = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual_h8w128.setImageHeight(8)
spectral_residual_h8w128.setImageWidth(128)
# h8w256
spectral_residual_h8w256 = cv2.saliency.StaticSaliencySpectralResidual_create()
spectral_residual_h8w256.setImageHeight(8)
spectral_residual_h8w256.setImageWidth(256)

# prepare baseline birth histogram / saliency
baseline_histogram_filename = "/data/datasets/nuscenes_results/object_birth_heatmaps/birth_heatmap_0849.npy"
baseline_birth_histogram = np.load(baseline_histogram_filename)
# make baseline horizontally symmetric
baseline_birth_histogram = (baseline_birth_histogram + np.fliplr(baseline_birth_histogram)) / 2
baseline_birth_histogram = baseline_birth_histogram / np.max(baseline_birth_histogram)
base_saliency = baseline_birth_histogram.copy()
baseline_birth_histogram_uint8 = (255 * baseline_birth_histogram).astype(np.uint8)
baseline_birth_histogram_heatmap = cv2.cvtColor(baseline_birth_histogram_uint8, cv2.COLOR_GRAY2BGR)
baseline_birth_histogram_heatmap = cv2.applyColorMap(baseline_birth_histogram_uint8, cv2.COLORMAP_JET)

# solo stats
base_stats = SaliencyEvalNewStats("base")
gbvs_stats = SaliencyEvalNewStats("gbvs")
spectralh64w64_stats = SaliencyEvalNewStats("spectralh64w64")
spectralh128w128_stats = SaliencyEvalNewStats("spectralh128w128")
spectralh8w64_stats = SaliencyEvalNewStats("spectralh8w64")
spectralh8w128_stats = SaliencyEvalNewStats("spectralh8w128")
spectralh8w256_stats = SaliencyEvalNewStats("spectralh8w256")
# gbvs combinations
gbvs_baseline_pwp_stats = SaliencyEvalNewStats("gbvs_baseline_pwp")
gbvs_baseline_ns_stats = SaliencyEvalNewStats("gbvs_baseline_ns")
gbvs_baseline_nm_stats = SaliencyEvalNewStats("gbvs_baseline_nm")
gbvs_baseline_cnws_stats = SaliencyEvalNewStats("gbvs_baseline_cnws")
# h64w64 combinations
spectralh64w64_baseline_pwp_stats = SaliencyEvalNewStats("spectralh64w64_baseline_pwp")
spectralh64w64_baseline_ns_stats = SaliencyEvalNewStats("spectralh64w64_baseline_ns")
spectralh64w64_baseline_nm_stats = SaliencyEvalNewStats("spectralh64w64_baseline_nm")
spectralh64w64_baseline_cnws_stats = SaliencyEvalNewStats("spectralh64w64_baseline_cnws")
# h128w128 combinations
spectralh128w128_baseline_pwp_stats = SaliencyEvalNewStats("spectralh128w128_baseline_pwp_stats")
spectralh128w128_baseline_ns_stats = SaliencyEvalNewStats("spectralh128w128_baseline_ns_stats")
spectralh128w128_baseline_nm_stats = SaliencyEvalNewStats("spectralh128w128_baseline_nm_stats")
spectralh128w128_baseline_cnws_stats = SaliencyEvalNewStats("spectralh128w128_baseline_cnws_stats")
# h8w64 combinations
spectralh8w64_baseline_pwp_stats = SaliencyEvalNewStats("spectralh8w64_baseline_pwp_stats")
spectralh8w64_baseline_ns_stats = SaliencyEvalNewStats("spectralh8w64_baseline_ns_stats")
spectralh8w64_baseline_nm_stats = SaliencyEvalNewStats("spectralh8w64_baseline_nm_stats")
spectralh8w64_baseline_cnws_stats = SaliencyEvalNewStats("spectralh8w64_baseline_cnws_stats")
# h8w128 combinations
spectralh8w128_baseline_pwp_stats = SaliencyEvalNewStats("spectralh8w128_baseline_pwp_stats")
spectralh8w128_baseline_ns_stats = SaliencyEvalNewStats("spectralh8w128_baseline_ns_stats")
spectralh8w128_baseline_nm_stats = SaliencyEvalNewStats("spectralh8w128_baseline_nm_stats")
spectralh8w128_baseline_cnws_stats = SaliencyEvalNewStats("spectralh8w128_baseline_cnws_stats")
# h8w256 combinations
spectralh8w256_baseline_pwp_stats = SaliencyEvalNewStats("spectralh8w256_baseline_pwp_stats")
spectralh8w256_baseline_ns_stats = SaliencyEvalNewStats("spectralh8w256_baseline_ns_stats")
spectralh8w256_baseline_nm_stats = SaliencyEvalNewStats("spectralh8w256_baseline_nm_stats")
spectralh8w256_baseline_cnws_stats = SaliencyEvalNewStats("spectralh8w256_baseline_cnws_stats")

timestamps: str = []
scene_idxs: int = []
image_filenames: str = []
categories: int = []

for idx_scene, scene in enumerate(nusc.scene):
    first_sample_rec = nusc.get('sample', scene['first_sample_token'])
    first_sd_rec = nusc.get('sample_data', first_sample_rec['data'][camera_channel])
    current_sd_rec = first_sd_rec
    print("evaluating scene ", idx_scene + 1)
    has_more_frames = True
    unique_instances = [] # list of all unique objects per scene
    while has_more_frames:
        # Get annotations, boxes, camera_intrinsic
        # When using BoxVisibility.ALL objects are detected too late in certain cases
        impath, boxes, camera_intrinsic = nusc.get_sample_data(current_sd_rec['token'], box_vis_level=BoxVisibility.ANY)
        if not os.path.exists(impath):
            raise Exception('Error: Missing image %s' % impath)

        current_im = cv2.imread(impath)
        current_im_box = current_im.copy()
        timestamp = current_sd_rec['timestamp']

        already_tracked_objects_mask = np.zeros((height, width), dtype=int)
        new_objects_mask = np.zeros((height, width), dtype=int)
        im_contains_unique_instance_and_is_not_first_frame = False
        new_object_categories_in_keyframe = []
        if current_sd_rec['is_key_frame']:
            for box in boxes:
                sample_annotation = nusc.get('sample_annotation', box.token)
                instance = nusc.get('instance', sample_annotation['instance_token'])
                category = nusc.get('category', instance['category_token'])
                # Exclude certain static objects, e.g. debris or traffic cones
                if category['index'] > 8 and category['index'] < 14:
                    continue
                # Only keep objects with certain visibility
                if int(sample_annotation['visibility_token']) < minimum_visibility:
                    continue
                skip_box_due_to_attribute = False
                for attribute_token in sample_annotation['attribute_tokens']:
                    attribute_name = nusc.get('attribute', attribute_token)['name']
                    if 'without_rider' in attribute_name or 'sitting' in attribute_name:
                        skip_box_due_to_attribute = True
                if skip_box_due_to_attribute:
                    continue
                new_object_categories_in_keyframe.append(category)
                # Gather unique instances
                instance_token = sample_annotation['instance_token']
                if instance_token not in unique_instances:
                    im_contains_unique_instance = True
                    unique_instances.append(instance_token)
                    if timestamp != first_sd_rec['timestamp']:
                        # This box can mean a potential object birth
                        im_contains_unique_instance_and_is_not_first_frame = True
                        # full bounding box
                        box_mask, _ = mask_pixels_from_box_cv2(box, width, height, view=camera_intrinsic, normalize=True)
                        
                        new_objects_mask = new_objects_mask + box_mask
                        box_render_cv2_text(box, nusc, current_im_box, view=camera_intrinsic, normalize=True, colors=(GREEN, GREEN, GREEN), render_text=False)
                        # this will be done in another evaluation
                        # new_objects_categories_and_masks.append(tuple([category, box_mask]))
                else:
                    box_mask, _ = mask_pixels_from_box_cv2(box, width, height, view=camera_intrinsic, normalize=True)
                    already_tracked_objects_mask = already_tracked_objects_mask + box_mask
                    # c = nusc.explorer.get_color(box.name)
                    box_render_cv2_text(box, nusc, current_im_box, view=camera_intrinsic, normalize=True,
                                        colors=(RED, RED, RED), render_text=False)

            # only evaluate frames that are keyframes, are not first frames and contain new unique instances 
            if im_contains_unique_instance_and_is_not_first_frame:
                timestamps.append(timestamp)
                scene_idxs.append(idx_scene + 1)
                image_filenames.append(impath)
                if len(new_object_categories_in_keyframe) > 1:
                    categories.append(42)
                else:
                    categories.append(category)
                # create mask usable in cv2
                new_objects_mask = new_objects_mask.clip(0, 255).astype(np.uint8)
                already_tracked_objects_mask = already_tracked_objects_mask.clip(0, 255).astype(np.uint8)
                if save_already_tracked:
                    already_tracked_filename = data_path + os.sep + "s{:04d}_{}_already_tracked.png".format(idx_scene + 1, timestamp)
                    cv2.imwrite(already_tracked_filename, already_tracked_objects_mask)
                # prepare different saliency base maps
                gbvs_filename = data_path + os.sep + "s{:04d}_{}_gbvs.exr".format(idx_scene + 1, timestamp)
                gbvs_saliency = cv2.imread(gbvs_filename, flags=(cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH))
                if gbvs_saliency is None:
                    print("Couldn't read gbvs saliency for ", gbvs_filename)
                (spectralh64w64_success, spectralh64w64_saliency) = spectral_residual_h64w64.computeSaliency(current_im)
                if not spectralh64w64_success:
                    print("Couldn't create saliency spectralh64w64 for ", impath)
                    continue
                (spectralh128w128_success, spectralh128w128_saliency) = spectral_residual_h128w128.computeSaliency(current_im)
                if not spectralh128w128_success:
                    print("Couldn't create saliency spectralh128w128 for ", impath)
                    continue
                (spectralh8w64_success, spectralh8w64_saliency) = spectral_residual_h8w64.computeSaliency(current_im)
                if not spectralh8w64_success:
                    print("Couldn't create saliency spectralh8w64 for ", impath)
                    continue
                (spectralh8w128_success, spectralh8w128_saliency) = spectral_residual_h8w128.computeSaliency(current_im)
                if not spectralh8w128_success:
                    print("Couldn't create saliency spectralh8w128 for ", impath)
                    continue
                (spectralh8w256_success, spectralh8w256_saliency) = spectral_residual_h8w256.computeSaliency(current_im)
                if not spectralh8w256_success:
                    print("Couldn't create saliency spectralh8w1256 for ", impath)
                    continue
                
                # create all gbvs combinations
                gbvs_base_pwp_saliency = simple_pixel_wise_product(gbvs_saliency, base_saliency)
                gbvs_base_ns_saliency = normalized_and_sum(gbvs_saliency, base_saliency)
                gbvs_base_nm_saliency = normalized_and_maximum(gbvs_saliency, base_saliency)
                gbvs_base_cnws_saliency = coherent_normalization_with_weighted_sum(gbvs_saliency, base_saliency)
                # create spectral h64w64 combinations
                spectralh64w64_base_pwp_saliency = simple_pixel_wise_product(spectralh64w64_saliency, base_saliency)
                spectralh64w64_base_ns_saliency = normalized_and_sum(spectralh64w64_saliency, base_saliency)
                spectralh64w64_base_nm_saliency = normalized_and_maximum(spectralh64w64_saliency, base_saliency)
                spectralh64w64_base_cnws_saliency = coherent_normalization_with_weighted_sum(spectralh64w64_saliency, base_saliency)
                # create spectral h128w128  combinations
                spectralh128w128_base_pwp_saliency = simple_pixel_wise_product(spectralh128w128_saliency, base_saliency)
                spectralh128w128_base_ns_saliency = normalized_and_sum(spectralh128w128_saliency, base_saliency)
                spectralh128w128_base_nm_saliency = normalized_and_maximum(spectralh128w128_saliency, base_saliency)
                spectralh128w128_base_cnws_saliency = coherent_normalization_with_weighted_sum(spectralh128w128_saliency, base_saliency)
                # create spectral h8w64  combinations
                spectralh8w64_base_pwp_saliency = simple_pixel_wise_product(spectralh8w64_saliency, base_saliency)
                spectralh8w64_base_ns_saliency = normalized_and_sum(spectralh8w64_saliency, base_saliency)
                spectralh8w64_base_nm_saliency = normalized_and_maximum(spectralh8w64_saliency, base_saliency)
                spectralh8w64_base_cnws_saliency = coherent_normalization_with_weighted_sum(spectralh8w64_saliency, base_saliency)
                # create spectral h8w128  combinations
                spectralh8w128_base_pwp_saliency = simple_pixel_wise_product(spectralh8w128_saliency, base_saliency)
                spectralh8w128_base_ns_saliency = normalized_and_sum(spectralh8w128_saliency, base_saliency)
                spectralh8w128_base_nm_saliency = normalized_and_maximum(spectralh8w128_saliency, base_saliency)
                spectralh8w128_base_cnws_saliency = coherent_normalization_with_weighted_sum(spectralh8w128_saliency, base_saliency)
                # create spectral h8w256  combinations
                spectralh8w256_base_pwp_saliency = simple_pixel_wise_product(spectralh8w256_saliency, base_saliency)
                spectralh8w256_base_ns_saliency = normalized_and_sum(spectralh8w256_saliency, base_saliency)
                spectralh8w256_base_nm_saliency = normalized_and_maximum(spectralh8w256_saliency, base_saliency)
                spectralh8w256_base_cnws_saliency = coherent_normalization_with_weighted_sum(spectralh8w256_saliency, base_saliency)
                
                # Calculate solo all errors
                mae_new_base, mae_im_base = mean_absolute_error_new(base_saliency, new_objects_mask)
                mae_new_gbvs, mae_im_gbvs = mean_absolute_error_new(gbvs_saliency, new_objects_mask)
                mae_new_spectralh64w64, mae_im_spectralh64w64 = mean_absolute_error_new(spectralh64w64_saliency, new_objects_mask)
                mae_new_spectralh128w128, mae_im_spectralh128w128 = mean_absolute_error_new(spectralh128w128_saliency, new_objects_mask)
                mae_new_spectralh8w64, mae_im_spectralh8w64 = mean_absolute_error_new(spectralh8w64_saliency, new_objects_mask)
                mae_new_spectralh8w128, mae_im_spectralh8w128 = mean_absolute_error_new(spectralh8w128_saliency, new_objects_mask)
                mae_new_spectralh8w256, mae_im_spectralh8w256 = mean_absolute_error_new(spectralh8w256_saliency, new_objects_mask)
                # calculate all gbvs combination errors
                mae_new_gbvs_base_pwp, mae_im_gbvs_base_pwp = mean_absolute_error_new(gbvs_base_pwp_saliency, new_objects_mask)
                mae_new_gbvs_base_ns, mae_im_gbvs_base_ns = mean_absolute_error_new(gbvs_base_ns_saliency, new_objects_mask)
                mae_new_gbvs_base_nm, mae_im_gbvs_base_nm = mean_absolute_error_new(gbvs_base_nm_saliency, new_objects_mask)
                mae_new_gbvs_base_cnws, mae_im_gbvs_base_cnws = mean_absolute_error_new(gbvs_base_cnws_saliency, new_objects_mask)
                # calculate all spectralh64w64 combination errors
                mae_new_spectralh64w64_base_pwp, mae_im_spectralh64w64_base_pwp = mean_absolute_error_new(spectralh64w64_base_pwp_saliency, new_objects_mask)
                mae_new_spectralh64w64_base_ns, mae_im_spectralh64w64_base_ns = mean_absolute_error_new(spectralh64w64_base_ns_saliency, new_objects_mask)
                mae_new_spectralh64w64_base_nm, mae_im_spectralh64w64_base_nm = mean_absolute_error_new(spectralh64w64_base_nm_saliency, new_objects_mask)
                mae_new_spectralh64w64_base_cnws, mae_im_spectralh64w64_base_cnws = mean_absolute_error_new(spectralh64w64_base_cnws_saliency, new_objects_mask)
                # calculate all spectralh8w128 combination errors
                mae_new_spectralh128w128_base_pwp, mae_im_spectralh128w128_base_pwp = mean_absolute_error_new(spectralh128w128_base_pwp_saliency, new_objects_mask)
                mae_new_spectralh128w128_base_ns, mae_im_spectralh128w128_base_ns = mean_absolute_error_new(spectralh128w128_base_ns_saliency, new_objects_mask)
                mae_new_spectralh128w128_base_nm, mae_im_spectralh128w128_base_nm = mean_absolute_error_new(spectralh128w128_base_nm_saliency, new_objects_mask)
                mae_new_spectralh128w128_base_cnws, mae_im_spectralh128w128_base_cnws = mean_absolute_error_new(spectralh128w128_base_cnws_saliency, new_objects_mask)
                # calculate all spectralh8w64 combination errors
                mae_new_spectralh8w64_base_pwp, mae_im_spectralh8w64_base_pwp = mean_absolute_error_new(spectralh8w64_base_pwp_saliency, new_objects_mask)
                mae_new_spectralh8w64_base_ns, mae_im_spectralh8w64_base_ns = mean_absolute_error_new(spectralh8w64_base_ns_saliency, new_objects_mask)
                mae_new_spectralh8w64_base_nm, mae_im_spectralh8w64_base_nm = mean_absolute_error_new(spectralh8w64_base_nm_saliency, new_objects_mask)
                mae_new_spectralh8w64_base_cnws, mae_im_spectralh8w64_base_cnws = mean_absolute_error_new(spectralh8w64_base_cnws_saliency, new_objects_mask)
                # calculate all spectralh8w128 combination errors
                mae_new_spectralh8w128_base_pwp, mae_im_spectralh8w128_base_pwp = mean_absolute_error_new(spectralh8w128_base_pwp_saliency, new_objects_mask)
                mae_new_spectralh8w128_base_ns, mae_im_spectralh8w128_base_ns = mean_absolute_error_new(spectralh8w128_base_ns_saliency, new_objects_mask)
                mae_new_spectralh8w128_base_nm, mae_im_spectralh8w128_base_nm = mean_absolute_error_new(spectralh8w128_base_nm_saliency, new_objects_mask)
                mae_new_spectralh8w128_base_cnws, mae_im_spectralh8w128_base_cnws = mean_absolute_error_new(spectralh8w128_base_cnws_saliency, new_objects_mask)
                # calculate all spectralh8w256 combination errors
                mae_new_spectralh8w256_base_pwp, mae_im_spectralh8w256_base_pwp = mean_absolute_error_new(spectralh8w256_base_pwp_saliency, new_objects_mask)
                mae_new_spectralh8w256_base_ns, mae_im_spectralh8w256_base_ns = mean_absolute_error_new(spectralh8w256_base_ns_saliency, new_objects_mask)
                mae_new_spectralh8w256_base_nm, mae_im_spectralh8w256_base_nm = mean_absolute_error_new(spectralh8w256_base_nm_saliency, new_objects_mask)
                mae_new_spectralh8w256_base_cnws, mae_im_spectralh8w256_base_cnws = mean_absolute_error_new(spectralh8w256_base_cnws_saliency, new_objects_mask)

                # add all solo errors
                base_stats.add_sample(mae_new_base, mae_im_base)
                gbvs_stats.add_sample(mae_new_gbvs, mae_im_gbvs)
                spectralh64w64_stats.add_sample(mae_new_spectralh64w64, mae_im_spectralh64w64)
                spectralh128w128_stats.add_sample(mae_new_spectralh128w128, mae_im_spectralh128w128)
                spectralh8w64_stats.add_sample(mae_new_spectralh8w64, mae_im_spectralh8w64)
                spectralh8w128_stats.add_sample(mae_new_spectralh8w128, mae_im_spectralh8w128)
                spectralh8w256_stats.add_sample(mae_new_spectralh8w256, mae_im_spectralh8w256)
                # add gbvs combination errors
                gbvs_baseline_pwp_stats.add_sample(mae_new_gbvs_base_pwp, mae_im_gbvs_base_pwp)
                gbvs_baseline_ns_stats.add_sample(mae_new_gbvs_base_ns, mae_im_gbvs_base_ns)
                gbvs_baseline_nm_stats.add_sample(mae_new_gbvs_base_nm, mae_im_gbvs_base_nm)
                gbvs_baseline_cnws_stats.add_sample(mae_new_gbvs_base_cnws, mae_im_gbvs_base_cnws)
                # add spectral h64w64 combination errors
                spectralh64w64_baseline_pwp_stats.add_sample(mae_new_spectralh64w64_base_pwp, mae_im_spectralh64w64_base_pwp)
                spectralh64w64_baseline_ns_stats.add_sample(mae_new_spectralh64w64_base_ns, mae_im_spectralh64w64_base_ns)
                spectralh64w64_baseline_nm_stats.add_sample(mae_new_spectralh64w64_base_nm, mae_im_spectralh64w64_base_nm)
                spectralh64w64_baseline_cnws_stats.add_sample(mae_new_spectralh64w64_base_cnws, mae_im_spectralh64w64_base_cnws)
                # add spectral h128w128 combination errors
                spectralh128w128_baseline_pwp_stats.add_sample(mae_new_spectralh128w128_base_pwp, mae_im_spectralh128w128_base_pwp)
                spectralh128w128_baseline_ns_stats.add_sample(mae_new_spectralh128w128_base_ns, mae_im_spectralh128w128_base_ns)
                spectralh128w128_baseline_nm_stats.add_sample(mae_new_spectralh128w128_base_nm, mae_im_spectralh128w128_base_nm)
                spectralh128w128_baseline_cnws_stats.add_sample(mae_new_spectralh128w128_base_cnws, mae_im_spectralh128w128_base_cnws)
                # add spectral h8w64 combination errors
                spectralh8w64_baseline_pwp_stats.add_sample(mae_new_spectralh8w64_base_pwp, mae_im_spectralh8w64_base_pwp)
                spectralh8w64_baseline_ns_stats.add_sample(mae_new_spectralh8w64_base_ns, mae_im_spectralh8w64_base_ns)
                spectralh8w64_baseline_nm_stats.add_sample(mae_new_spectralh8w64_base_nm, mae_im_spectralh8w64_base_nm)
                spectralh8w64_baseline_cnws_stats.add_sample(mae_new_spectralh8w64_base_cnws, mae_im_spectralh8w64_base_cnws)
                # add spectral h8w128 combination errors
                spectralh8w128_baseline_pwp_stats.add_sample(mae_new_spectralh8w128_base_pwp, mae_im_spectralh8w128_base_pwp)
                spectralh8w128_baseline_ns_stats.add_sample(mae_new_spectralh8w128_base_ns, mae_im_spectralh8w128_base_ns)
                spectralh8w128_baseline_nm_stats.add_sample(mae_new_spectralh8w128_base_nm, mae_im_spectralh8w128_base_nm)
                spectralh8w128_baseline_cnws_stats.add_sample(mae_new_spectralh8w128_base_cnws, mae_im_spectralh8w128_base_cnws)
                # add spectral h8w256 combination errors
                spectralh8w256_baseline_pwp_stats.add_sample(mae_new_spectralh8w256_base_pwp, mae_im_spectralh8w256_base_pwp)
                spectralh8w256_baseline_ns_stats.add_sample(mae_new_spectralh8w256_base_ns, mae_im_spectralh8w256_base_ns)
                spectralh8w256_baseline_nm_stats.add_sample(mae_new_spectralh8w256_base_nm, mae_im_spectralh8w256_base_nm)
                spectralh8w256_baseline_cnws_stats.add_sample(mae_new_spectralh8w256_base_cnws, mae_im_spectralh8w256_base_cnws)
                
                if debug_single:
                    debug_im_and_saliency(current_im_box, base_saliency, "base_saliency", mae_new_base, mae_im_base)
                    debug_im_and_saliency(current_im_box, gbvs_saliency, "gbvs_saliency", mae_new_gbvs, mae_im_gbvs)
                    debug_im_and_saliency(current_im_box, spectralh64w64_saliency, "spectral_residual_h64w64", mae_new_spectralh64w64, mae_im_spectralh64w64)
                    debug_im_and_saliency(current_im_box, spectralh128w128_saliency, "spectral_residual_h128w128", mae_new_spectralh128w128, mae_im_spectralh128w128)
                    debug_im_and_saliency(current_im_box, spectralh8w64_saliency, "spectral_residual_h8w64", mae_new_spectralh8w64, mae_im_spectralh8w64)
                    debug_im_and_saliency(current_im_box, spectralh8w128_saliency, "spectral_residual_h8w128", mae_new_spectralh8w128, mae_im_spectralh8w128)
                    debug_im_and_saliency(current_im_box, spectralh8w256_saliency, "spectral_residual_h8w256", mae_new_spectralh8w256, mae_im_spectralh8w256)
                    # # display gbvs combinations
                    debug_im_and_saliency(current_im_box, gbvs_base_pwp_saliency, "gbvs_base_pwp_saliency", mae_new_gbvs_base_pwp, mae_im_gbvs_base_pwp)
                    debug_im_and_saliency(current_im_box, gbvs_base_ns_saliency, "gbvs_base_ns_saliency", mae_new_gbvs_base_ns, mae_im_gbvs_base_ns)
                    debug_im_and_saliency(current_im_box, gbvs_base_nm_saliency, "gbvs_base_nm_saliency", mae_new_gbvs_base_nm, mae_im_gbvs_base_nm)
                    debug_im_and_saliency(current_im_box, gbvs_base_cnws_saliency, "gbvs_base_cnws_saliency", mae_new_gbvs_base_cnws, mae_im_gbvs_base_cnws)
                    # # display spectral combinations
                    debug_im_and_saliency(current_im_box, spectralh64w64_base_pwp_saliency, "spectralh64w64_base_pwp_saliency", mae_new_spectralh64w64_base_pwp, mae_im_spectralh64w64_base_pwp)
                    debug_im_and_saliency(current_im_box, spectralh64w64_base_ns_saliency, "spectralh64w64_base_ns_saliency", mae_new_spectralh64w64_base_ns, mae_im_spectralh64w64_base_ns)
                    debug_im_and_saliency(current_im_box, spectralh64w64_base_nm_saliency, "spectralh64w64_base_nm_saliency", mae_new_spectralh64w64_base_nm, mae_im_spectralh64w64_base_nm)
                    debug_im_and_saliency(current_im_box, spectralh64w64_base_cnws_saliency, "spectralh64w64_base_cnws_saliency", mae_new_spectralh64w64_base_cnws, mae_im_spectralh64w64_base_cnws)
                    # display spectral combinations
                    debug_im_and_saliency(current_im_box, spectralh128w128_base_pwp_saliency, "spectralh128w128_base_pwp_saliency", mae_new_spectralh128w128_base_pwp, mae_im_spectralh128w128_base_pwp)
                    debug_im_and_saliency(current_im_box, spectralh128w128_base_ns_saliency, "spectralh128w128_base_ns_saliency", mae_new_spectralh128w128_base_ns, mae_im_spectralh128w128_base_ns)
                    debug_im_and_saliency(current_im_box, spectralh128w128_base_nm_saliency, "spectralh128w128_base_nm_saliency", mae_new_spectralh128w128_base_nm, mae_im_spectralh128w128_base_nm)
                    debug_im_and_saliency(current_im_box, spectralh128w128_base_cnws_saliency, "spectralh128w128_base_cnws_saliency", mae_new_spectralh128w128_base_cnws, mae_im_spectralh128w128_base_cnws)
                    # display spectral combinations
                    debug_im_and_saliency(current_im_box, spectralh8w64_base_pwp_saliency, "spectralh8w64_base_pwp_saliency", mae_new_spectralh8w64_base_pwp, mae_im_spectralh8w64_base_pwp)
                    debug_im_and_saliency(current_im_box, spectralh8w64_base_ns_saliency, "spectralh8w64_base_ns_saliency", mae_new_spectralh8w64_base_ns, mae_im_spectralh8w64_base_ns)
                    debug_im_and_saliency(current_im_box, spectralh8w64_base_nm_saliency, "spectralh8w64_base_nm_saliency", mae_new_spectralh8w64_base_nm, mae_im_spectralh8w64_base_nm)
                    debug_im_and_saliency(current_im_box, spectralh8w64_base_cnws_saliency, "spectralh8w64_base_cnws_saliency", mae_new_spectralh8w64_base_cnws, mae_im_spectralh8w64_base_cnws)
                    # display spectral combinations
                    debug_im_and_saliency(current_im_box, spectralh8w128_base_pwp_saliency, "spectralh8w128_base_pwp_saliency", mae_new_spectralh8w128_base_pwp, mae_im_spectralh8w128_base_pwp)
                    debug_im_and_saliency(current_im_box, spectralh8w128_base_ns_saliency, "spectralh8w128_base_ns_saliency", mae_new_spectralh8w128_base_ns, mae_im_spectralh8w128_base_ns)
                    debug_im_and_saliency(current_im_box, spectralh8w128_base_nm_saliency, "spectralh8w128_base_nm_saliency", mae_new_spectralh8w128_base_nm, mae_im_spectralh8w128_base_nm)
                    debug_im_and_saliency(current_im_box, spectralh8w128_base_cnws_saliency, "spectralh8w128_base_cnws_saliency", mae_new_spectralh8w128_base_cnws, mae_im_spectralh8w128_base_cnws)
                    # display spectral combinations
                    debug_im_and_saliency(current_im_box, spectralh8w256_base_pwp_saliency, "spectralh8w256_base_pwp_saliency", mae_new_spectralh8w256_base_pwp, mae_im_spectralh8w256_base_pwp)
                    debug_im_and_saliency(current_im_box, spectralh8w256_base_ns_saliency, "spectralh8w256_base_ns_saliency", mae_new_spectralh8w256_base_ns, mae_im_spectralh8w256_base_ns)
                    debug_im_and_saliency(current_im_box, spectralh8w256_base_nm_saliency, "spectralh8w256_base_nm_saliency", mae_new_spectralh8w256_base_nm, mae_im_spectralh8w256_base_nm)
                    debug_im_and_saliency(current_im_box, spectralh8w256_base_cnws_saliency, "spectralh8w256_base_cnws_saliency", mae_new_spectralh8w256_base_cnws, mae_im_spectralh8w256_base_cnws)

                if debug:
                    new_objects_mask = cv2.cvtColor(new_objects_mask, cv2.COLOR_GRAY2BGR)
                    # debug all images
                    indices_already_tracked_objects = np.where(already_tracked_objects_mask == 255)
                    already_tracked_objects_mask = cv2.cvtColor(already_tracked_objects_mask, cv2.COLOR_GRAY2BGR)
                    if len(indices_already_tracked_objects[0]) > 0:
                        already_tracked_objects_mask[indices_already_tracked_objects[0], indices_already_tracked_objects[1]] = (0, 0, 255)
                    # already_tracked_objects_mask = cv2.cvtColor(already_tracked_objects_mask, cv2.COLOR_GRAY2BGR)

                    display_im = cv2.addWeighted(current_im, 0.7, new_objects_mask, 0.3, 0.0)
                    display_im = cv2.addWeighted(display_im, 0.7, already_tracked_objects_mask, 0.3, 0.0)
                    cv2.imshow("im, new and tracked objects", display_im)
                    cv2.imshow("already tracked objects", already_tracked_objects_mask)
                    if im_contains_unique_instance_and_is_not_first_frame:
                        key = cv2.waitKey(0)
                    else:
                        key = cv2.waitKey(1)

                    if key == 27:
                        break_all = True
                        break
                    
        if break_all:
            break
        
        if current_sd_rec['next'] == '':
            has_more_frames = False
        else:
            current_sd_rec = nusc.get('sample_data', current_sd_rec['next'])
        
    if break_all:
        break

cv2.destroyAllWindows()


evaluating scene  1


/home/serov/anaconda3/envs/nuscenes_local/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  app.launch_new_instance()


In [171]:
# MAE_NEW_ONLY -> bounding box of new object compared to saliency map cutout
# MAE_WHOLE_IMAGE -> bounding box of new object and whole image compared to whole saliency map
# MAE_TRACKED_OBJECTS_ONLY -> bounding box of already tracked objects compared to saliency map cutout
# MAE_TRACKED_OBJECTS_WHOLE_IMAGE -> bounding box of already tracked objects and whole image compared to saliency map


# try out spectralh8w256 for splitting birth areas
# protoobjects see spectral residual paper

In [174]:
# save pickle objects
import pickle
base_pickle_path = "/data/datasets/nuscenes_results/eval_keyframes_only_new1"

def save_stats(stats, name):
    with open(base_pickle_path + os.sep + name + ".pkl", 'wb') as outp:
        pickle.dump(stats, outp, pickle.HIGHEST_PROTOCOL)
        


In [175]:
save_stats(timestamps, "timestamps")
save_stats(scene_idxs, "scene_idxs")
save_stats(image_filenames, "image_filenames")
save_stats(categories, "categories")
# single
save_stats(base_stats, "base_stats")
save_stats(gbvs_stats, "gbvs_stats")
save_stats(spectralh64w64_stats, "spectralh64w64_stats")
save_stats(spectralh128w128_stats, "spectralh128w128_stats")
save_stats(spectralh8w64_stats, "spectralh8w64_stats")
save_stats(spectralh8w128_stats, "spectralh8w128_stats")
save_stats(spectralh8w256_stats, "spectralh8w256_stats")
# gbvs combi
save_stats(gbvs_baseline_pwp_stats, "gbvs_baseline_pwp_stats")
save_stats(gbvs_baseline_ns_stats, "gbvs_baseline_ns_stats")
save_stats(gbvs_baseline_nm_stats, "gbvs_baseline_nm_stats")
save_stats(gbvs_baseline_cnws_stats, "gbvs_baseline_cnws_stats")
# spectralh64w64 combi
save_stats(spectralh64w64_baseline_pwp_stats, "spectralh64w64_baseline_pwp_stats")
save_stats(spectralh64w64_baseline_ns_stats, "spectralh64w64_baseline_ns_stats")
save_stats(spectralh64w64_baseline_nm_stats, "spectralh64w64_baseline_nm_stats")
save_stats(spectralh64w64_baseline_cnws_stats, "spectralh64w64_baseline_cnws_stats")
# spectralh128w128 combi
save_stats(spectralh128w128_baseline_pwp_stats, "spectralh128w128_baseline_pwp_stats")
save_stats(spectralh128w128_baseline_ns_stats, "spectralh128w128_baseline_ns_stats")
save_stats(spectralh128w128_baseline_nm_stats, "spectralh128w128_baseline_nm_stats")
save_stats(spectralh128w128_baseline_cnws_stats, "spectralh128w128_baseline_cnws_stats")
# spectralh8w64 combi
save_stats(spectralh8w64_baseline_pwp_stats, "spectralh8w64_baseline_pwp_stats")
save_stats(spectralh8w64_baseline_ns_stats, "spectralh8w64_baseline_ns_stats")
save_stats(spectralh8w64_baseline_nm_stats, "spectralh8w64_baseline_nm_stats")
save_stats(spectralh8w64_baseline_cnws_stats, "spectralh8w64_baseline_cnws_stats")
# spectralh8w128 combi
save_stats(spectralh8w128_baseline_pwp_stats, "spectralh8w128_baseline_pwp_stats")
save_stats(spectralh8w128_baseline_ns_stats, "spectralh8w128_baseline_ns_stats")
save_stats(spectralh8w128_baseline_nm_stats, "spectralh8w128_baseline_nm_stats")
save_stats(spectralh8w128_baseline_cnws_stats, "spectralh8w128_baseline_cnws_stats")
# spectralh8w256 combi
save_stats(spectralh8w256_baseline_pwp_stats, "spectralh8w256_baseline_pwp_stats")
save_stats(spectralh8w256_baseline_ns_stats, "spectralh8w256_baseline_ns_stats")
save_stats(spectralh8w256_baseline_nm_stats, "spectralh8w256_baseline_nm_stats")
save_stats(spectralh8w256_baseline_cnws_stats, "spectralh8w256_baseline_cnws_stats")


In [176]:
# print stats
base_stats.print_stats()
gbvs_stats.print_stats()
spectralh64w64_stats.print_stats()
spectralh128w128_stats.print_stats()
spectralh8w64_stats.print_stats()
spectralh8w128_stats.print_stats()
spectralh8w256_stats.print_stats()
gbvs_baseline_pwp_stats.print_stats()
gbvs_baseline_ns_stats.print_stats()
gbvs_baseline_nm_stats.print_stats()
gbvs_baseline_cnws_stats.print_stats()
spectralh64w64_baseline_pwp_stats.print_stats()
spectralh64w64_baseline_ns_stats.print_stats()
spectralh64w64_baseline_nm_stats.print_stats()
spectralh64w64_baseline_cnws_stats.print_stats()
spectralh128w128_baseline_pwp_stats.print_stats()
spectralh128w128_baseline_ns_stats.print_stats()
spectralh128w128_baseline_nm_stats.print_stats()
spectralh128w128_baseline_cnws_stats.print_stats()
spectralh8w64_baseline_pwp_stats.print_stats()
spectralh8w64_baseline_ns_stats.print_stats()
spectralh8w64_baseline_nm_stats.print_stats()
spectralh8w64_baseline_cnws_stats.print_stats()
spectralh8w128_baseline_pwp_stats.print_stats()
spectralh8w128_baseline_ns_stats.print_stats()
spectralh8w128_baseline_nm_stats.print_stats()
spectralh8w128_baseline_cnws_stats.print_stats()
spectralh8w256_baseline_pwp_stats.print_stats()
spectralh8w256_baseline_ns_stats.print_stats()
spectralh8w256_baseline_nm_stats.print_stats()
spectralh8w256_baseline_cnws_stats.print_stats()


base                           n: 2  mae_new 0.4035987381  std_new  0.0248553979  mae_tracked  0.0844349499  std_tracked  0.0016639734
gbvs                           n: 2  mae_new 0.8689432949  std_new  0.0299471913  mae_tracked  0.1686851563  std_tracked  0.0006110894
spectralh64w64                 n: 2  mae_new 0.8674382029  std_new  0.0153462375  mae_tracked  0.0999057726  std_tracked  0.0042432834
spectralh128w128               n: 2  mae_new 0.9235159120  std_new  0.0063390437  mae_tracked  0.0575329400  std_tracked  0.0062305474
spectralh8w64                  n: 2  mae_new 0.9181954066  std_new  0.0074269175  mae_tracked  0.1247022786  std_tracked  0.0096881402
spectralh8w128                 n: 2  mae_new 0.9181954066  std_new  0.0074269175  mae_tracked  0.1247022786  std_tracked  0.0096881402
spectralh8w256                 n: 2  mae_new 0.9320089910  std_new  0.0264112910  mae_tracked  0.1106496935  std_tracked  0.0279455431
gbvs_baseline_pwp              n: 2  mae_new 0.71018084

In [177]:
# def print_stats_nicely(stats: SaliencyEvalStats):
#     print("{:30} n: {:d}  mae_new {:0.10f}  std_new  {:0.10f}  mae_tracked  {:0.10f}  std_tracked  {:0.10f}".format(
#         stats.name, stats.n, np.mean(np.asarray(stats.mean_absolute_error_new_objects)), np.std(np.asarray(stats.mean_absolute_error_new_objects)),
#         np.mean(np.asarray(stats.mean_absolute_error_already_tracked_objects)), np.std(np.asarray(stats.mean_absolute_error_already_tracked_objects))))

# print_stats_nicely(base_stats)
# print_stats_nicely(gbvs_stats)
# print_stats_nicely(spectralh8w128_stats)
# print_stats_nicely(spectralh64w64_stats)
# print_stats_nicely(spectralh128w128_stats)
# print_stats_nicely(gbvs_baseline_pwp_stats)
# print_stats_nicely(gbvs_baseline_ns_stats)
# print_stats_nicely(gbvs_baseline_nm_stats)
# print_stats_nicely(gbvs_baseline_cnws_stats)
# print_stats_nicely(spectralh64w64_baseline_pwp_stats)
# print_stats_nicely(spectralh64w64_baseline_ns_stats)
# print_stats_nicely(spectralh64w64_baseline_nm_stats)
# print_stats_nicely(spectralh64w64_baseline_cnws_stats)


In [178]:
# import glob

# # TODO save MAE stats for every sample to find outliers and investigate distribution of data

# width = 1600
# height = 900
# display_image = True
# save_images = True
# # get all images and birth masks
# base_data_path = "/data/datasets/nuscenes_results/birth_frames_and_masks"

# image_filenames = sorted(glob.glob(base_data_path + os.sep + "*image*", recursive=False))
# mask_filenames = sorted(glob.glob(base_data_path + os.sep + "*mask*", recursive=False))
# gbvs_filenames = sorted(glob.glob(base_data_path + os.sep + "*gbvs*", recursive=False))

# assert(len(image_filenames) == len(mask_filenames) and len(gbvs_filenames) == len(mask_filenames))

# # create spectral residual object
# spectral_residual_height = 8
# spectral_residual_width = 256
# spectral_residual = cv2.saliency.StaticSaliencySpectralResidual_create()
# spectral_residual.setImageHeight(spectral_residual_height)
# spectral_residual.setImageWidth(spectral_residual_width)

# # prepare object birth histogram
# baseline_histogram_filename = "/data/datasets/nuscenes_results/object_birth_heatmaps/birth_heatmap_0849.npy"
# baseline_birth_histogram = np.load(baseline_histogram_filename)
# baseline_birth_histogram = normalize(baseline_birth_histogram)

# # saliency maps to be evaluated
# saliency_types = ['baseline', 'gbvs', 'spectral_residual_h8w256']
# # Combinations?
# saliency_results = {}
# for saliency_type in saliency_types:
#     saliency_results[saliency_type] = RunningStats()

# for image_filename, mask_filename, gbvs_filename in zip(image_filenames, mask_filenames, gbvs_filenames):
#     img = cv2.imread(image_filename)
#     mask = cv2.imread(mask_filename, flags=cv2.IMREAD_GRAYSCALE)
#     gbvs = cv2.imread(gbvs_filename, flags=(cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH))
#     (success, saliency_spectral_residual) = spectral_residual.computeSaliency(img)
#     if not success:
#         print("Couldn't create saliency for ", image_filename)
#         continue
    
#     img_and_gbvs = overlay_image_and_saliency_as_heatmap(img, gbvs, 1.0)
#     img_and_baseline = overlay_image_and_saliency_as_heatmap(img, baseline_birth_histogram, 1.0)
#     # saliency_spectral_residual = cv2.cvtColor((saliency_spectral_residual*255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
#     img_and_spectral_residual = overlay_image_and_saliency_as_heatmap(img, normalize(saliency_spectral_residual), 1.0)

#     mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
#     img_and_gbvs_and_mask = cv2.addWeighted(img_and_gbvs, 0.7, mask_bgr, 0.3, 0.0)
#     img_and_baseline_and_mask = cv2.addWeighted(img_and_baseline, 0.7, mask_bgr, 0.3, 0.0)
#     img_and_spectral_residual_and_mask = cv2.addWeighted(img_and_spectral_residual, 0.7, mask_bgr, 0.3, 0.0)

#     mae_baseline = mean_absolute_error(baseline_birth_histogram, mask)
#     mae_gbvs = mean_absolute_error(gbvs, mask)
#     mae_spectral_residual = mean_absolute_error(saliency_spectral_residual, mask)#

#     saliency_results['baseline'].add_sample(mae_baseline)
#     saliency_results['gbvs'].add_sample(mae_gbvs)
#     saliency_results['spectral_residual_h8w256'].add_sample(mae_spectral_residual)

#     mae_baseline = mean_absolute_error(baseline_birth_histogram, mask)
#     cv2_put_multi_line_text(img_and_gbvs_and_mask, str(mae_gbvs), (width/2, height/2), (0, 0, 0))
#     cv2_put_multi_line_text(img_and_baseline_and_mask, str(mae_baseline), (width/2, height/2), (0, 0, 0))
#     cv2.imshow("input img and gbvs overlayed", img_and_spectral_residual_and_mask)
#     key = cv2.waitKey(0)
#     print("MEAN base {} gbvs {} spectral {}".format(
#         saliency_results['baseline'].mean(),
#         saliency_results['gbvs'].mean(),
#         saliency_results['spectral_residual_h8w256'].mean()))
#     print("STD  base {} gbvs {} spectral {}\n".format(
#         saliency_results['baseline'].standard_deviation(),
#         saliency_results['gbvs'].standard_deviation(),
#         saliency_results['spectral_residual_h8w256'].standard_deviation()))
#     if key == 27:  # if ESC is pressed, exit.
#         cv2.destroyAllWindows()
#         break

# cv2.destroyAllWindows()
    


In [179]:
# convex mask code
# sample_record = nusc.get('sample', current_sd_rec['sample_token'])
# sample_token = current_sd_rec['sample_token']
# pointsensor_token = sample_record['data'][pointsensor_channel]
# camera_token = sample_record['data'][camera_channel]
# lidar_points_inside_box, coloring = map_pointcloud_to_box(nusc, sample_token, pointsensor_token, camera_token, box)
# lidar_points_inside_box = lidar_points_inside_box[:2, :]
# convex_mask, convex_hull_valid, hull_points = convex_mask_and_hull_points_from_lidar_points(
#     box_mask, lidar_points_inside_box, width, height)
# indices_contour = np.where(convex_mask == 255)
# convex_mask = cv2.cvtColor(convex_mask, cv2.COLOR_GRAY2BGR)
# convex_mask[indices_contour[0], indices_contour[1]] = (0, 0, 255)
# final_im = cv2.addWeighted(final_im, 0.7, convex_mask, 0.3, 0.0)
# cv2_put_multi_line_text(final_im, str(convex_hull_valid), (width/2, height/2), color=(255, 0, 0))
# for p in lidar_points_inside_box.T:
#     cv2.drawMarker(final_im, (p[0].astype(int), p[1].astype(int)), color=(
#         0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=20)
